# Welcome to Semantic Kernel with Microsoft Fabric - NLP to SQL
- This notebook has following examples:
    - install semantic-kernel library in Microsoft Fabric Spark environment
    - Run Skills inline, directly in the code
    - Run Skills from a file in the LakeHouse
    - stream text/chat completions
- Todo: Secrets are hardcoded in this notebook for simplicity for the workshop - to move to Azure Keyvault
- Prerequisites:
    - Access to Azure OpenAI, and values for the endpoint, the key, the deployments
    - Upload Skills to the Lakehouse 

In [50]:
#By adding library in workspace setting, you have already installed semantic-kernel! 
#in case you have not setup in workspace setting, then uncomment below line and run this cell
#
#%pip install semantic-kernel --quiet

%pip show semantic-kernel

StatementMeta(, 5eb134e0-1a40-4c46-b0da-4369bd275f6c, 54, Finished, Available)

Name: semantic-kernel
Version: 0.3.12.dev0
Summary: 
Home-page: 
Author: Microsoft
Author-email: SK-Support@microsoft.com
License: 
Location: /home/trusted-service-user/cluster-env/trident_env/lib/python3.10/site-packages
Requires: aiofiles, numpy, openai, openapi_core, prance, pydantic, python-dotenv, regex
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [51]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai import ChatRequestSettings, CompleteRequestSettings
from semantic_kernel.connectors.ai.open_ai import AzureTextCompletion, AzureChatCompletion

StatementMeta(, 5eb134e0-1a40-4c46-b0da-4369bd275f6c, 55, Finished, Available)

In [52]:
api_key = "*****"
endpoint = "https://******.openai.azure.com/"
azure_text_service = AzureTextCompletion("text-davinci-003", endpoint, api_key)
azure_chat_service = AzureChatCompletion("gpt-35-turbo", endpoint, api_key)

StatementMeta(, 5eb134e0-1a40-4c46-b0da-4369bd275f6c, 56, Finished, Available)

In [53]:
kernel = sk.Kernel()
kernel.add_text_completion_service("text_completion", azure_text_service)

StatementMeta(, 5eb134e0-1a40-4c46-b0da-4369bd275f6c, 57, Finished, Available)

## Example - NLP to SQL

In [54]:
skills_directory = "/lakehouse/default/Files/sk-samples-skills"
nlp2sqlFunction = kernel.import_semantic_skill_from_directory(skills_directory, "nlp2sqlSkill")["GenSQL"]

StatementMeta(, 5eb134e0-1a40-4c46-b0da-4369bd275f6c, 58, Finished, Available)

In [55]:
nlp_query = "What is the total Sales Quantity last year?"
#
result = await nlp2sqlFunction.invoke_async(nlp_query)
print(result)

StatementMeta(, 5eb134e0-1a40-4c46-b0da-4369bd275f6c, 59, Finished, Available)


SELECT SUM(SalesQuantity) 
FROM FactSales 
WHERE YEAR(DateKey) = YEAR(DATEADD(YEAR, -1, GETDATE()));


In [56]:
nlp_query = "How many store types are there"
#
result = await nlp2sqlFunction.invoke_async(nlp_query)
print(result)

StatementMeta(, 5eb134e0-1a40-4c46-b0da-4369bd275f6c, 60, Finished, Available)


SELECT COUNT(DISTINCT StoreType) 
FROM DimStore;


In [57]:
nlp_query = "What is the total Sales Quantity last year for OPENMALL store type?"
#
result = await nlp2sqlFunction.invoke_async(nlp_query)
print(result)

StatementMeta(, 5eb134e0-1a40-4c46-b0da-4369bd275f6c, 61, Finished, Available)


SELECT SUM(SalesQuantity) 
FROM FactSales 
INNER JOIN DimStore 
ON FactSales.StoreKey = DimStore.StoreKey 
WHERE DimStore.StoreType = 'OPENMALL' 
AND YEAR(FactSales.DateKey) = YEAR(CURRENT_TIMESTAMP) - 1;


In [58]:
nlp_query = "Give me total Sales Quantity and Sales Amount by store type and year"
#
result = await nlp2sqlFunction.invoke_async(nlp_query)
print(result)

StatementMeta(, 5eb134e0-1a40-4c46-b0da-4369bd275f6c, 62, Finished, Available)


SELECT 
	StoreType, 
	YEAR(DateKey) AS Year, 
	SUM(SalesQuantity) AS TotalSalesQuantity, 
	SUM(SalesAmount) AS TotalSalesAmount 
FROM FactSales 
INNER JOIN DimStore 
	ON FactSales.StoreKey = DimStore.StoreKey 
GROUP BY StoreType, YEAR(DateKey) 
ORDER BY StoreType, Year;


In [59]:
nlp_query = "Give me total Sales Quantity and Sales Amount by year, store type and region"
#
result = await nlp2sqlFunction.invoke_async(nlp_query)
print(result)

StatementMeta(, 5eb134e0-1a40-4c46-b0da-4369bd275f6c, 63, Finished, Available)


SELECT YEAR(s.LoadDate) AS Year, 
	   ds.StoreType, 
	   ds.GeographyKey, 
	   SUM(s.SalesQuantity) AS TotalSalesQuantity, 
	   SUM(s.SalesAmount) AS TotalSalesAmount
FROM FactSales s
INNER JOIN DimStore ds
	ON s.StoreKey = ds.StoreKey
GROUP BY YEAR(s.LoadDate), ds.StoreType, ds.GeographyKey;


## End of this Notebook

In [60]:
#Sample Schema
#https://github.com/microsoft/sql-server-samples/blob/master/samples/databases/contoso-data-warehouse/load-contoso-data-warehouse-to-sql-data-warehouse.sql

StatementMeta(, 5eb134e0-1a40-4c46-b0da-4369bd275f6c, 64, Finished, Available)